In [1]:
# importando as libs
import pandas as pd
import numpy as np
import requests as req
from selenium import webdriver
from scrapy.selector import Selector as scp
import json

In [2]:
dsm = pd.read_csv('musica_tophits_00.htm.csv', delimiter=';', header=None)
dsm

,0,1
0,Amor I Love You,Marisa Monte (EMI)
1,Se Eu Não Te Amasse Tanto Assim,Ivete Sangalo (Universal)
2,Smooth,Santana feat. Rob Thomas (BMG)
3,Xibom Bombom,As Meninas (Universal)
4,Tô Te Filmando (Sorria),Os Travessos (Continental)
5,What A Girl Wants,Christina Aguilera (BMG)
6,Anna Julia,Los Hermanos (Abril)
7,Music,Madonna (Warner)
8,El Arbi,Khaled (Universal)
9,Minha Razão de Viver,Ara Ketu (Sony)


In [4]:
#df_final = pd.DataFrame(columns=["Posição", "Musica", "Artista", "Gênero", "Quantidade de palavras únicas", "Quantidade de acordes", "Quantidade de exibições", "Link da letra", "Link da cifra"])

i = 0

for linha in dsm.iterrows():
    i += 1
    str_de_busca = "'{} {}' site:www.letras.mus.br".format(linha[1][0], linha[1][1])
    str_de_busca = str_de_busca.replace("&","")
    print(str_de_busca)
    driver = webdriver.PhantomJS()
    try:
        driver.get("https://duckduckgo.com/?q=" + str_de_busca)
        #print(scp(text=driver.page_source).xpath('//a[@class="result__a"]/@href').extract())
        buscando_link = scp(text=driver.page_source).xpath('//a[@class="result__a"]/@href').extract_first()
        #print(buscando_link)
        buscando_link = buscando_link.replace("traducao.html","")
    except:
        continue
    r = req.get(buscando_link)
    if r.status_code == 200:
        texto = r.content
        #print(texto)
        letra_aux = scp(text=texto).xpath('//div[@class="cnt-letra p402_premium"]/article/p/text()').extract()
        qtd_palavras_unicas = len(list(set(' '.join(letra_aux).lower().split(' '))))
        #print(scp(text=texto).xpath('//div[@class="cnt-info_exib"]/b/text()').extract_first())
        qtd_exibicoes = scp(text=texto).xpath('//div[@class="cnt-info_exib"]/b/text()').extract_first()
        qtd_exibicoes = int(qtd_exibicoes.replace('.',''))
        genero = scp(text=texto).xpath('//div[@id="breadcrumb"]/span/a/span/text()').extract()[1]
        links_a_esquerda = scp(text=texto).xpath('//div[@class="letra-menu"]/a/@href').extract()
        link_da_cifra = links_a_esquerda[2]
        if links_a_esquerda[0] == "#":
            link_da_cifra = links_a_esquerda[1]
        if qtd_palavras_unicas > 1:
            #print(link_da_cifra)
            rc = req.get(link_da_cifra)
            if rc.status_code == 200:
                texto = rc.content
                #print(texto)
                acordes = scp(text=texto).xpath('//div[@id="js-view-script"]/script/text()').extract()[1]
                inicio = acordes.find('chords: ') + 8
                fim = acordes.find('}]);') - 14
                acordes = acordes[inicio:fim]
                js = json.loads(acordes)
                chords = []
                for chord in js:
                    chords.append(chord['chord'])                        
                #novo_df = pd.DataFrame([i, linha[1][0], linha[1][1], genero, qtd_palavras_unicas, len(chords), qtd_exibicoes, buscando_link, link_da_cifra], columns=["Posição", "Musica", "Artista", "Gênero", "Quantidade de palavras únicas", "Quantidade de acordes", "Quantidade de exibições", "Link da letra", "Link da cifra"])
                #df_final.append(novo_df, ignore_index=True)
                print("{}, {}, {}, {}, {}\n".format(linha[1][0], qtd_palavras_unicas, qtd_exibicoes, genero, len(chords)))
            else:
                print("\n")
    else:
        print("Não achou link")

'Amor I Love You Marisa Monte (EMI)' site:www.letras.mus.br


/mnt/hd/Estudos/Python/cientifico/musical/lib/python3.6/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '


Amor I Love You, 110, 1987325, MPB, 7

'Se Eu Não Te Amasse Tanto Assim Ivete Sangalo (Universal)' site:www.letras.mus.br


/mnt/hd/Estudos/Python/cientifico/musical/lib/python3.6/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '


Se Eu Não Te Amasse Tanto Assim, 69, 2263659, Axé, 17

'Smooth Santana feat. Rob Thomas (BMG)' site:www.letras.mus.br


...

Smooth, 105, 82598, Pop, 0

'Xibom Bombom As Meninas (Universal)' site:www.letras.mus.br


/mnt/hd/Estudos/Python/cientifico/musical/lib/python3.6/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '


Xibom Bombom, 64, 274206, Axé, 6

'Tô Te Filmando (Sorria) Os Travessos (Continental)' site:www.letras.mus.br


/mnt/hd/Estudos/Python/cientifico/musical/lib/python3.6/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '


JSONDecodeError: Expecting value: line 1 column 11 (char 10)